# Data 전처리 

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2018)

# load data
data_path = '/Users/seonghoonjung/.kaggle/data/santander-product-recommendation'
trn = pd.read_csv(f'{data_path}/train_ver2.csv')
tst = pd.read_csv(f'{data_path}/test_ver2.csv')

# 제품 변수 저장
prods = trn.columns[24:].tolist()

# 제품 변수 결측값 => 0
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 모든 제품을 다 보유하지 않은 사용자 데이터는 삭제
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

# train/test 데이터를 통합하여 전처리를 해야 한다. 
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

features = []

# 범주형 데이터는 label encoding
cat_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in cat_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += cat_cols

# 수치형 변수는 특이값과 결측값을 -99로 대체하고 정수형으로 변환 
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

/Users/seonghoonjung/anaconda3/envs/kaggle/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/seonghoonjung/anaconda3/envs/kaggle/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 피처 엔지니어링

* 두 개의 날자 문자열에서 연도/달 추출
* lag-1 추가 

In [2]:
# (피쳐 엔지니어링) 두 날짜 변수에서 연도와 월 정보를 추출한다.
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8) # month
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16) # year
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8) # month
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16) # year
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

# 그 외 변수의 결측값은 모두 -99로 대체한다.
df.fillna(-99, inplace=True)

# (피쳐 엔지니어링) lag-1 데이터를 생성한다.
# 코드 2-12와 유사한 코드 흐름이다.

# 날짜를 숫자로 변환하는 함수이다. 2015-01-28은 1, 2016-06-28은 18로 변환된다
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

# 날짜를 숫자로 변환하여 int_date에 저장한다
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

# 데이터를 복사하고, int_date 날짜에 1을 더하여 lag를 생성한다. 변수명에 _prev를 추가한다.
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns ]
df_lag['int_date'] += 1

# 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 합친다. Lag 데이터의 int_date는 1 밀려 있기 때문에, 저번 달의 제품 정보가 삽입된다.
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')

# 메모리 효율을 위해 불필요한 변수를 메모리에서 제거한다
del df, df_lag

# 저번 달의 제품 정보가 존재하지 않을 경우를 대비하여 0으로 대체한다.
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

# lag-1 변수를 추가한다.
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

In [5]:
df_trn.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_tjcr_fin_ult1_prev,ind_valo_fin_ult1_prev,ind_viv_fin_ult1_prev,ind_nomina_ult1_prev,ind_nom_pens_ult1_prev,ind_recibo_ult1_prev,fecha_alta_month_prev,fecha_alta_year_prev,ult_fec_cli_1t_month_prev,ult_fec_cli_1t_year_prev
0,2015-01-28,1375586,0,0,0,35,2015-01-12,0.0,6,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0
1,2015-01-28,1050611,0,0,1,23,2012-08-10,0.0,35,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0
2,2015-01-28,1050612,0,0,1,23,2012-08-10,0.0,35,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0
3,2015-01-28,1050613,0,0,0,22,2012-08-10,0.0,35,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0
4,2015-01-28,1050614,0,0,1,23,2012-08-10,0.0,35,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-99.0,-99.0,-99.0,-99.0


# 교차 검증


In [3]:
# 학습에 2016년도 4개월치(1~4) 데이터만 사용, 검증 데이터로 5월달 데이터 사용, 테스트 데이터는 6월달 데이터 

use_dates = ['2016-01-28','2016-02-28','2016-03-28','2016-04-28','2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']

In [4]:
# 훈련 데이터에는 신규 구매 건수만 추출 (lag-1 활용), 즉 신규 구매한 고객 데이터만으로 필터링
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod]==1) & (trn[prev]==0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

# 검증 데이터 분리 
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]


# 모델 학습

In [ ]:
# XGBoost 모델 parameter를 설정한다.
param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
    }

# 훈련, 검증 데이터를 XGBoost 형태로 변환한다.
X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

# XGBoost 모델을 훈련 데이터로 학습한다!
#watch_list = [(dtrn, 'train'), (dvld, 'eval')]
watch_list = [(dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=500, evals=watch_list, early_stopping_rounds=20)

# 학습한 모델을 저장한다.
import pickle
pickle.dump(model, open("xgb.baseline.pkl", "wb"))
best_ntree_limit = model.best_ntree_limit


# 평가

In [5]:
def apk(actual, predicted, k=7, default=0.0):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    
    for i, p in enumerate(predicted):
        # 예측값이 정답 리스트에 있고, 중복이 아니면 hit
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score = num_hits / (i+1.0)
    
    # 정답이 공백인 경우 0 반환
    if not actual:
        return default
    
    # 정답 개수로 average precision 구하기 
    return score / min(len(actual), k)

def mapk(actual, predicted, k=7, default=0.0):
    # list of list = 고객.제품
    return np.mean([ apk(a,p,k,default) for a,p in zip(actual, predicted)])

apk([1,2,3,0,0,0,0], [3,2,1,0,0,0,0])

0.14285714285714285

In [6]:
# 고객 식별 번호 추출
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld.as_matrix(columns=['ncodpers'])

# 검증 데이터에서 신규 구매 label을 구한다. 
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev]
add_vld = vld.as_matrix(columns=[prod + '_add' for prod in prods])  # 고객.제품 -> 신규 구매 여부, ex) 홍길동 -> 1, 0, 0, 1, 0, .... 0 

# 고객.제품별 신규 구매 정답값을 list에 저장하고, 총 count를 저장 (즉 신규 구매건만 기억하고, 아닌 것은 제거), ex) 홍길동 -> prod_1, prod_4
add_vld_list = [list() for i in range(len(ncodpers_vld))]
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] >0:
            add_vld_list[ncodper].append(prod)
            count_vld +=1

# 검증 데이터에서 얻을 수 있는 MAP@7의 최고점을 구한다 (0.042663)
print(mapk(add_vld_list, add_vld_list, 7, 0.0))


    
    


/Users/seonghoonjung/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/seonghoonjung/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/seonghoonjung/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.


0.03762404785183121


In [8]:
# 학습한 모델을 저장한다.
import pickle

predict_data_size = 1000

# load
model = pickle.load(open("xgb.baseline.pkl", "rb"))
best_ntree_limit = model.best_ntree_limit

# 검증 데이터에 대해 예측값을 구한다. 
X_vld = vld.as_matrix(columns=features)[:predict_data_size]
Y_vld = vld.as_matrix(columns=['y'])[:predict_data_size]
dvld  = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)

# 저번 달에 보유한 제품은 신규 구매가 불가하므로, 확률값에서 미리 1을 빼준다. 
preds_vld = preds_vld - vld.as_matrix(columns=[prod + '_prev' for prod in prods])[:predict_data_size]

# 검증 데이터 예측 상위 7개 추출
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y,p,ip in y_prods])

# 검증 데이터에서의 MAP@7 점수를 구한다. 
print(mapk(add_vld_list, result_vld, 7, 0.0))

/Users/seonghoonjung/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/Users/seonghoonjung/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


0.024792857142857144


/Users/seonghoonjung/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
